In [46]:
import json
import requests
from openai import OpenAI
client = OpenAI()
# client = OpenAI(api_key="sk-acc46ad3186942bd9840998a39de8f4f", base_url="https://api.deepseek.com")

with open('questions_and_queries.json', 'r', encoding='utf-8') as file:
    example_data = json.load(file)

with open('annotations_2.json', 'r', encoding='utf-8') as file: #annotation_2 is what I named the new annotations file
    label_data = json.load(file)

    
def generate_sparql_query(user_question, label_data, examples_data):
    concepts_and_labels = "\n".join(
        f"URI of Label: {pair['uri']}\nLabel: {pair['label']}\n" for pair in label_data
    )
    example_queries = "\n".join(
        f"User Question: {pair['user_question']}\nSPARQL Query: {pair['sparql_query']}\n" for pair in examples_data
    )

    prompt = f"""
    GIVE ONLY THE QUERY AS AN ANSWER TO THE FOLLOWING PROMPT:

    The following are all the labels for the decisions in the decisions dataset and their URIs:

    {concepts_and_labels}

    Based on the user's question: {user_question}, go through all the labels then choose one label that best matches the question's theme and context. 
    
    After you chose the label, look at the question again: {user_question}, and go ahead and choose a different label than the one chosen in the previous step. Compare the two labels, and choose only the one best fitting the question but show me both.
    
    Get the URI of the chosen label, these will be used in the next step.

    NEXT STEP:

    The following are examples of user questions in Dutch and their corresponding SPARQL queries:

    {example_queries}

    Based on the examples above and the URIs of the chosen labels, generate a SPARQL query for the following user question:

    User Question: {user_question}
    SPARQL Query:

    But please make sure to use the URIs of the chosen labels in the "?annotation oa:hasBody" part of the query like in the examples. 
    
    If there is more than one URI, you can separate them with a comma. 
    
    Make sure to not include any extra spaces or '\n' characters in the query.

    Then after filtering on label, add a filter for the title or description or motivering with keywords that you extract from the question.
    """
    response = client.chat.completions.create(
            # model="deepseek-chat",
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that generates SPARQL queries to be run on a SPARQL endpoint containing data about decisions of the city of Gent based on user questions and labels of decisions related to their questions. Your language is Dutch."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0
        )

    return response

# Example user question
# user_question = "Wat waren de laatste 3 besluiten betreffende de middelbare school in Gent?"
user_question = "Where can I go swimming?"

# Generate SPARQL query for the example user question
sparql_query = generate_sparql_query(user_question, label_data=label_data, examples_data=example_data)
print(f"User Question: {user_question}")
print(f"Generated SPARQL Query: {sparql_query}")

User Question: Where can I go swimming?
Generated SPARQL Query: ChatCompletion(id='chatcmpl-9lcIHrbcVnJBXUV1sfPlFNzV6056a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="PREFIX prov:<http://www.w3.org/ns/prov#> PREFIX besluit:<http://data.vlaanderen.be/ns/besluit#> PREFIX eli:<http://data.europa.eu/eli/ontology#> PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX oa:<http://www.w3.org/ns/oa#> SELECT ?annotation ?target ?title ?motivering (GROUP_CONCAT(?value;separator=', ') AS ?values) ?description ?date ?derivedFrom WHERE { ?annotation oa:hasBody <http://stad.gent/id/concepts/gent_words/485> ; oa:hasTarget ?target . ?target eli:title ?title ; besluit:motivering ?motivering ; prov:value ?value ; eli:description ?description ; eli:date_publication ?date ; prov:wasDerivedFrom ?derivedFrom . FILTER (CONTAINS(LCASE(?title),'zwemmen') || CONTAINS(LCASE(?description),'zwemmen') || CONTAINS(LCASE(?motivering),'zwemmen'))}", r

In [47]:
query_content = sparql_query.choices[0].message.content

prefix_position = query_content.find("PREFIX")
if prefix_position != -1:
    query_content = query_content[prefix_position:]

query_content_no_newlines = query_content.replace("\n", " ")

endpoint_url = "https://probe.stad.gent/sparql"
headers = {
    "Accept": "application/sparql-results+json"
}
params = {
    "query": query_content_no_newlines
}

response = requests.get(endpoint_url, headers=headers, params=params)
if response.status_code == 200:
    results = response.json()
    print(f"Results for question '{user_question}':", results)
else:
    print(f"Failed to execute query for question '{user_question}':", response.status_code)

Results for question 'Where can I go swimming?': {'results': {'ordered': True, 'distinct': False, 'bindings': [{'values': {'value': '\n                                                \n        \n            Artikel 1\n            \n            \n                De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen.\n            \n        \n        \n    \n                                            , \n                                    \n        \n            Artikel 1\n            \n            \n                De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen k

In [23]:
query_content_no_newlines

"PREFIX prov: <http://www.w3.org/ns/prov#> \nPREFIX besluit: <http://data.vlaanderen.be/ns/besluit#> \nPREFIX eli: <http://data.europa.eu/eli/ontology#> \nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> \nPREFIX oa: <http://www.w3.org/ns/oa#> \nSELECT ?annotation ?target ?title ?motivering (GROUP_CONCAT(?value; separator=', ') AS ?values) ?description ?date ?derivedFrom \nWHERE { \n  ?annotation oa:hasBody <http://stad.gent/id/concepts/gent_words/485> ; \n  oa:hasTarget ?target . \n  ?target eli:title ?title ; \n  besluit:motivering ?motivering ; \n  prov:value ?value ; \n  eli:description ?description ; \n  eli:date_publication ?date ; \n  prov:wasDerivedFrom ?derivedFrom . \n  FILTER (CONTAINS(LCASE(?title), 'zwemmen') || CONTAINS(LCASE(?description), 'zwemmen')) \n}"

In [48]:
results_content = results['results']['bindings']

In [21]:
results_content

[{'values': {'value': '\n                                                \n        \n            Artikel 1\n            \n            \n                Keurt goed de amendering van het gemeenteraadsbesluit betreffende de wijziging aan het reglement \'Retributie voor diensten van administratieve en technische aard van het Departement Onderwijs, Opvoeding en Jeugd\', geagendeerd op de gemeenteraad van mei 2022, in die zin dat:\xa0een aangepaste versie van de gecoördineerde versies van het reglement als bijlage bij het gemeenteraadsbesluit wordt gevoegd ter vervanging van de oorspronkelijke bijlageneen artikel wordt tussengevoegd dat luidt als volgt:Vervangt artikel 2, §1 d van het Reglement \'Retributie voor diensten van administratieve en technische aard van het Departement Onderwijs, Opvoeding en Jeugd\' zoals goedgekeurd door de gemeenteraad van 18 december 2019 als volgt:"d. Het inschrijvingsgeld in het deeltijds kunstonderwijs voor houders van een UiTPAS met kansentarief per schoolj

In [49]:
import re
cleaned_decisions = []
resources = []

for decision in results_content:
    cleaned_decision = {}
    for key, detail in decision.items():
        # Extract the value and remove any \n or extra spaces
        cleaned_value = re.sub(r'\s+', ' ', detail['value']).strip()
        cleaned_decision[key] = cleaned_value
    cleaned_decisions.append(cleaned_decision)

# Print or use the cleaned_decisions as needed
for d in cleaned_decisions:
    print(d)

for d in cleaned_decisions:
    resources.append(d['derivedFrom'])

{'values': 'Artikel 1 De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen. , Artikel 1 De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen.', 'title': '2022_VVB_00015 - Geamendeerd voorstel tot raadsbesluit van raadslid Anneleen Van Bossuyt: Zwemmen aan de Blaarmeersen', 'target': 'https://data.gent.be/id/besluiten/22.0518.8364.2858', 'motivering': 'Motivering Wat gaat aan deze beslissing vooraf Tijdens het weekend van 14-15 mei was zwemmen aan de Blaarmeersen ondanks de hoge temperaturen niet mogelijk. Het bijzonder goede weer wa

In [50]:
prompt_2 = f"""
  The following is the question the user asked:

  {user_question}

  Based on the following data only, generate a response that answers their question, don't hallucinate:

  Data: {cleaned_decisions}
  Answer in the language the user asked in. Be sure to be friendly and explain in a way an ordinary person can understand. The language city officials use is often very different from the language citizens use (officials wind up speaking in departments and form numbers instead of needs in big organizations).

  Show the {resources} to the end-user so they can refer to them if they want.
  """

completion_2 = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant for the citizens of Gent when they ask a question on the city's website regarding the decisions made by the city. Be friendly, speak in easy to use terms. You use both the user's question and relevant decisions passed to you."},
    {"role": "user", "content": prompt_2}
  ]
)

print(completion_2.choices[0].message.content)

I see you're interested in finding a place to go swimming in Gent. Based on the recent decisions, the city council is looking into allowing swimming at Blaarmeersen earlier in the season on warm days. 

You can get more information about this decision and stay updated by checking the following links:
- [Decision 1](https://ebesluitvorming.gent.be/zittingen/21.1004.3056.7946/notulen)
- [Decision 2](https://ebesluitvorming.gent.be/zittingen/21.1004.3056.7946/agendapunten/22.0523.4375.3376)
- [Decision 3](https://ebesluitvorming.gent.be/zittingen/21.1004.3056.7946/notulen)
- [Decision 4](https://ebesluitvorming.gent.be/zittingen/21.1004.3056.7946/notulen)
- [Decision 5](https://ebesluitvorming.gent.be/zittingen/21.1004.3056.7946/agendapunten/22.0518.8364.2858)
- [Decision 6](https://ebesluitvorming.gent.be/zittingen/21.1004.3056.7946/agendapunten/22.0523.4375.3376)
- [Decision 7](https://ebesluitvorming.gent.be/zittingen/21.1004.3056.7946/notulen)
- [Decision 8](https://ebesluitvorming.ge